In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT
import numpy as np
from datetime import datetime

# CONFIGURACIÓN INICIAL 
carpeta = r"C:\Users\Fer\OneDrive\Escritorio\Modelado de Mineria de Datos\Proyecto_3\supermercado"

archivos_csv = glob.glob(os.path.join(carpeta, "*.csv"))

if not archivos_csv:
    print(" No se encontraron archivos CSV en la carpeta especificada.")
else:
    print(f" Se encontraron {len(archivos_csv)} archivos CSV:")
    for a in archivos_csv:
        print(" -", os.path.basename(a))

# Crear carpetas de salida
carpeta_excels = os.path.join(carpeta, "Excels_Individuales")
carpeta_graficos_combinados = os.path.join(carpeta, "Graficos_Combinados")
carpeta_informe_combinado = os.path.join(carpeta, "Informe_Combinado")

os.makedirs(carpeta_excels, exist_ok=True)
os.makedirs(carpeta_graficos_combinados, exist_ok=True)
os.makedirs(carpeta_informe_combinado, exist_ok=True)

# Archivo combinado final
salida_excel_combinado = os.path.join(carpeta, "Datos_Combinados.xlsx")

# --- FUNCIÓN PARA CARGAR TABLAS CON FORMATO CORRECTO ---
def cargar_tablas(archivos_csv):
    """
    Carga todos los CSV con los formatos correctos basados en las imágenes
    """
    dataframes = {}
    
    for archivo in archivos_csv:
        nombre = os.path.splitext(os.path.basename(archivo))[0]
        print(f"\n Cargando {nombre}...")
        
        try:
            # Cargar el CSV
            df = pd.read_csv(archivo)
            print(f"   Columnas originales: {list(df.columns)}")
            print(f"   Primeras filas:")
            print(df.head(3))
            
            # Limpiar nombres de columnas (eliminar comillas y espacios)
            df.columns = df.columns.str.replace('"', '').str.strip()
            
            # Procesamiento específico para cada tabla basado en las imágenes
            if 'cliente' in nombre.lower() and 'mail' not in nombre.lower() and 'telefono' not in nombre.lower():
                # Tabla cliente principal
                if len(df.columns) >= 3:
                    df.columns = ['id_cliente', 'nombre', 'apellido', 'direccion'] if len(df.columns) == 4 else ['id_cliente', 'nombre_completo', 'direccion']
                print(f"   Tabla cliente procesada")
                
            elif 'cliente_mail' in nombre.lower():
                # Tabla cliente_mail
                if len(df.columns) >= 2:
                    df.columns = ['id_email', 'id_cliente', 'email'] if len(df.columns) == 3 else ['id_cliente', 'email']
                print(f"   Tabla cliente_mail procesada")
                
            elif 'cliente_telefono' in nombre.lower():
                # Tabla cliente_telefono
                if len(df.columns) >= 2:
                    df.columns = ['id_tel', 'id_cliente', 'telefono', 'alternativo'] if len(df.columns) == 4 else ['id_cliente', 'telefono']
                print(f"   Tabla cliente_telefono procesada")
                
            elif 'factura_detalle' in nombre.lower():
                # Tabla factura_detalle
                if len(df.columns) >= 4:
                    df.columns = ['id_detalle', 'id_factura', 'id_producto', 'cantidad', 'precio_unitario']
                print(f"   Tabla factura_detalle procesada")
                
            elif 'factura_enunciado' in nombre.lower() or 'factura_encabezado' in nombre.lower():
                # Tabla factura_enunciado
                if len(df.columns) >= 3:
                    df.columns = ['id_factura', 'numero_factura', 'fecha', 'id_venta']
                print(f"   Tabla factura_enunciado procesada")
                
            elif 'producto' in nombre.lower():
                # Tabla producto
                if len(df.columns) >= 3:
                    df.columns = ['id_producto', 'nombre_producto', 'precio', 'id_rubro']
                print(f"   Tabla producto procesada")
                
            elif 'rubro' in nombre.lower():
                # Tabla rubros
                if len(df.columns) >= 1:
                    df.columns = ['id_rubro', 'nombre_rubro']
                print(f"   Tabla rubros procesada")
                
            elif 'sucursal' in nombre.lower():
                # Tabla sucursal
                if len(df.columns) >= 2:
                    df.columns = ['id_sucursal', 'nombre_sucursal', 'direccion']
                print(f"   Tabla sucursal procesada")
                
            elif 'venta' in nombre.lower():
                # Tabla venta
                if len(df.columns) >= 3:
                    df.columns = ['id_venta', 'id_cliente', 'id_sucursal', 'fecha']
                print(f"   Tabla venta procesada")
                
            elif 'localidad' in nombre.lower():
                # Tabla localidad
                if len(df.columns) >= 2:
                    df.columns = ['id_localidad', 'nombre_localidad', 'id_provincia']
                print(f"   Tabla localidad procesada")
                
            elif 'provincia' in nombre.lower():
                # Tabla provincia
                if len(df.columns) >= 1:
                    df.columns = ['id_provincia', 'nombre_provincia']
                print(f"   Tabla provincia procesada")
            
            dataframes[nombre] = df
            print(f"   Columnas finales: {list(df.columns)}")
            print(f"   Filas: {len(df)}")
            
        except Exception as e:
            print(f"   ERROR cargando {nombre}: {e}")
    
    return dataframes

# --- FUNCIÓN PARA GENERAR GRÁFICOS COMBINADOS ---
def generar_graficos_combinados(dataframes, carpeta_graficos):
    """
    Genera gráficos específicos combinando las tablas correctamente
    """
    graficos_generados = []
    
    print("\n=== IDENTIFICANDO TABLAS ===")
    
    # Identificar tablas por nombre
    cliente_df = None
    factura_detalle_df = None
    factura_enunciado_df = None
    producto_df = None
    rubros_df = None
    venta_df = None
    
    for nombre, df in dataframes.items():
        if 'cliente' in nombre.lower() and 'mail' not in nombre.lower() and 'telefono' not in nombre.lower():
            cliente_df = df
            print(f"✓ Identificada tabla cliente: {len(df)} registros")
            
        elif 'factura_detalle' in nombre.lower():
            factura_detalle_df = df
            print(f"✓ Identificada tabla factura_detalle: {len(df)} registros")
            
        elif 'factura_enunciado' in nombre.lower() or 'factura_encabezado' in nombre.lower():
            factura_enunciado_df = df
            print(f"✓ Identificada tabla factura_enunciado: {len(df)} registros")
            
        elif 'producto' in nombre.lower():
            producto_df = df
            print(f"✓ Identificada tabla producto: {len(df)} registros")
            
        elif 'rubro' in nombre.lower():
            rubros_df = df
            print(f"✓ Identificada tabla rubros: {len(df)} registros")
            
        elif 'venta' in nombre.lower():
            venta_df = df
            print(f"✓ Identificada tabla venta: {len(df)} registros")
    
    # Verificar que tenemos las tablas mínimas necesarias
    if factura_detalle_df is None or producto_df is None:
        print(" ERROR: No se encontraron las tablas mínimas necesarias (factura_detalle y producto)")
        return graficos_generados
    
    print("\n=== GENERANDO GRÁFICOS ===")
    
    try:
        # 1. LOS 10 PRODUCTOS MÁS VENDIDOS
        print("1. Generando Top 10 productos más vendidos...")
        
        # Calcular productos más vendidos por cantidad
        productos_vendidos = factura_detalle_df.groupby('id_producto')['cantidad'].sum().nlargest(10)
        
        # Unir con productos para obtener nombres
        if producto_df is not None:
            productos_vendidos_con_nombre = productos_vendidos.reset_index().merge(
                producto_df[['id_producto', 'nombre_producto']], 
                on='id_producto', 
                how='left'
            )
            
            # Gráfico de barras
            plt.figure(figsize=(12, 6))
            plt.bar(range(len(productos_vendidos_con_nombre)), productos_vendidos_con_nombre['cantidad'])
            plt.title('Top 10 Productos Más Vendidos por Cantidad')
            plt.xlabel('Productos')
            plt.ylabel('Cantidad Vendida')
            plt.xticks(range(len(productos_vendidos_con_nombre)), productos_vendidos_con_nombre['nombre_producto'], rotation=45, ha='right')
            plt.tight_layout()
            path_barras = os.path.join(carpeta_graficos, "01_top10_productos_barras.png")
            plt.savefig(path_barras)
            plt.close()
            graficos_generados.append(path_barras)
            
            # Gráfico de torta
            plt.figure(figsize=(10, 8))
            plt.pie(productos_vendidos_con_nombre['cantidad'], labels=productos_vendidos_con_nombre['nombre_producto'], autopct='%1.1f%%')
            plt.title('Distribución de Top 10 Productos Más Vendidos')
            plt.tight_layout()
            path_torta = os.path.join(carpeta_graficos, "02_top10_productos_torta.png")
            plt.savefig(path_torta)
            plt.close()
            graficos_generados.append(path_torta)
        
        # 2. VENTAS TOTALES POR RUBRO
        print("2. Generando ventas por rubro...")
        
        if rubros_df is not None:
            # Unir factura_detalle con productos para obtener rubros
            detalle_con_rubro = factura_detalle_df.merge(
                producto_df[['id_producto', 'id_rubro']], 
                on='id_producto', 
                how='left'
            ).merge(
                rubros_df[['id_rubro', 'nombre_rubro']], 
                on='id_rubro', 
                how='left'
            )
            
            ventas_por_rubro = detalle_con_rubro.groupby('nombre_rubro')['cantidad'].sum()
            
            plt.figure(figsize=(12, 6))
            ventas_por_rubro.plot(kind='bar', color='skyblue')
            plt.title('Ventas Totales por Rubro')
            plt.xlabel('Rubro')
            plt.ylabel('Cantidad Vendida')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            path_barras = os.path.join(carpeta_graficos, "03_ventas_por_rubro_barras.png")
            plt.savefig(path_barras)
            plt.close()
            graficos_generados.append(path_barras)
        
        # 3. VENTAS POR MES
        print("3. Generando ventas por mes...")
        
        if factura_enunciado_df is not None:
            # Convertir fecha y agrupar por mes
            factura_enunciado_df['fecha'] = pd.to_datetime(factura_enunciado_df['fecha'])
            ventas_por_mes = factura_enunciado_df.groupby(factura_enunciado_df['fecha'].dt.to_period('M')).size()
            
            plt.figure(figsize=(12, 6))
            ventas_por_mes.plot(kind='line', marker='o', color='green')
            plt.title('Ventas por Mes')
            plt.xlabel('Mes')
            plt.ylabel('Cantidad de Ventas')
            plt.xticks(rotation=45)
            plt.tight_layout()
            path_lineal = os.path.join(carpeta_graficos, "04_ventas_por_mes_lineal.png")
            plt.savefig(path_lineal)
            plt.close()
            graficos_generados.append(path_lineal)
        
        # 4. TICKET PROMEDIO
        print("4. Generando ticket promedio...")
        
        if factura_detalle_df is not None and factura_enunciado_df is not None and venta_df is not None and cliente_df is not None:
            # Calcular total por factura
            factura_detalle_df['total_linea'] = factura_detalle_df['cantidad'] * factura_detalle_df['precio_unitario']
            total_por_factura = factura_detalle_df.groupby('id_factura')['total_linea'].sum()
            
            # Unir con factura_enunciado y venta para obtener cliente
            ticket_info = total_por_factura.reset_index().merge(
                factura_enunciado_df[['id_factura', 'id_venta']],
                on='id_factura',
                how='left'
            ).merge(
                venta_df[['id_venta', 'id_cliente']],
                on='id_venta',
                how='left'
            )
            
            # Calcular ticket promedio por cliente
            ticket_promedio = ticket_info.groupby('id_cliente')['total_linea'].mean().nlargest(10)
            
            plt.figure(figsize=(12, 6))
            ticket_promedio.plot(kind='bar', color='orange')
            plt.title('Ticket Promedio por Cliente (Top 10)')
            plt.xlabel('ID Cliente')
            plt.ylabel('Ticket Promedio ($)')
            plt.xticks(rotation=45)
            plt.tight_layout()
            path_barras = os.path.join(carpeta_graficos, "05_ticket_promedio_barras.png")
            plt.savefig(path_barras)
            plt.close()
            graficos_generados.append(path_barras)
        
        # 5. FACTURA MÁS ALTA
        print("5. Generando facturas más altas...")
        
        if factura_detalle_df is not None:
            # Calcular total por factura
            factura_detalle_df['total_linea'] = factura_detalle_df['cantidad'] * factura_detalle_df['precio_unitario']
            total_por_factura = factura_detalle_df.groupby('id_factura')['total_linea'].sum().nlargest(10)
            
            plt.figure(figsize=(12, 6))
            total_por_factura.plot(kind='bar', color='red')
            plt.title('Top 10 Facturas Más Altas')
            plt.xlabel('ID Factura')
            plt.ylabel('Monto Total ($)')
            plt.xticks(rotation=45)
            plt.tight_layout()
            path_barras = os.path.join(carpeta_graficos, "06_facturas_mas_altas_barras.png")
            plt.savefig(path_barras)
            plt.close()
            graficos_generados.append(path_barras)
        
        # 6. PRODUCTO MÁS VENDIDO EN CANTIDAD
        print("6. Generando producto más vendido...")
        
        producto_mas_vendido = factura_detalle_df.groupby('id_producto')['cantidad'].sum().nlargest(5)
        
        if producto_df is not None:
            producto_mas_vendido_con_nombre = producto_mas_vendido.reset_index().merge(
                producto_df[['id_producto', 'nombre_producto']],
                on='id_producto',
                how='left'
            )
            
            plt.figure(figsize=(10, 6))
            plt.bar(producto_mas_vendido_con_nombre['nombre_producto'], producto_mas_vendido_con_nombre['cantidad'])
            plt.title('Top 5 Productos Más Vendidos en Cantidad')
            plt.xlabel('Producto')
            plt.ylabel('Cantidad Vendida')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            path_barras = os.path.join(carpeta_graficos, "07_producto_mas_vendido_barras.png")
            plt.savefig(path_barras)
            plt.close()
            graficos_generados.append(path_barras)
        
        # 7. TOTAL DE VENTAS POR CLIENTE
        print("7. Generando ventas por cliente...")
        
        if factura_detalle_df is not None and factura_enunciado_df is not None and venta_df is not None:
            # Calcular total por factura
            factura_detalle_df['total_linea'] = factura_detalle_df['cantidad'] * factura_detalle_df['precio_unitario']
            total_por_factura = factura_detalle_df.groupby('id_factura')['total_linea'].sum()
            
            # Unir para obtener cliente
            ventas_por_cliente = total_por_factura.reset_index().merge(
                factura_enunciado_df[['id_factura', 'id_venta']],
                on='id_factura',
                how='left'
            ).merge(
                venta_df[['id_venta', 'id_cliente']],
                on='id_venta',
                how='left'
            )
            
            ventas_totales_cliente = ventas_por_cliente.groupby('id_cliente')['total_linea'].sum().nlargest(10)
            
            plt.figure(figsize=(12, 6))
            ventas_totales_cliente.plot(kind='line', marker='o', color='purple')
            plt.title('Total de Ventas por Cliente (Top 10)')
            plt.xlabel('ID Cliente')
            plt.ylabel('Total de Ventas ($)')
            plt.xticks(rotation=45)
            plt.tight_layout()
            path_lineal = os.path.join(carpeta_graficos, "08_ventas_por_cliente_lineal.png")
            plt.savefig(path_lineal)
            plt.close()
            graficos_generados.append(path_lineal)
        
        # 8. UNIÓN VENTA-CLIENTE (Cantidad de ventas por cliente)
        print("8. Generando unión venta-cliente...")
        
        if venta_df is not None:
            ventas_por_cliente_count = venta_df.groupby('id_cliente').size().nlargest(10)
            
            plt.figure(figsize=(12, 6))
            ventas_por_cliente_count.plot(kind='bar', color='lightgreen')
            plt.title('Cantidad de Ventas por Cliente (Top 10)')
            plt.xlabel('ID Cliente')
            plt.ylabel('Cantidad de Ventas')
            plt.xticks(rotation=45)
            plt.tight_layout()
            path_barras = os.path.join(carpeta_graficos, "09_ventas_por_cliente_cantidad_barras.png")
            plt.savefig(path_barras)
            plt.close()
            graficos_generados.append(path_barras)
            
    except Exception as e:
        print(f" ERROR generando gráficos: {e}")
        import traceback
        traceback.print_exc()
    
    return graficos_generados

# --- FUNCIÓN PARA CREAR INFORME COMBINADO ---
def crear_informe_combinado(graficos_combinados, carpeta_informe, dataframes):
    """
    Crea un PDF con todos los gráficos combinados generados
    """
    pdf_path = os.path.join(carpeta_informe, "Informe_Combinado_Completo.pdf")
    doc = SimpleDocTemplate(pdf_path, pagesize=letter)
    styles = getSampleStyleSheet()
    
    estilo_titulo = ParagraphStyle(
        name="Titulo",
        parent=styles["Title"],
        fontSize=20,
        alignment=TA_CENTER,
        spaceAfter=30,
    )
    
    estilo_subtitulo = ParagraphStyle(
        name="Subtitulo",
        parent=styles["Heading2"],
        fontSize=14,
        alignment=TA_LEFT,
        spaceAfter=12,
    )
    
    story = []
    
    # Portada
    story.append(Spacer(1, 200))
    story.append(Paragraph("INFORME COMBINADO DE ANÁLISIS", estilo_titulo))
    story.append(Spacer(1, 20))
    story.append(Paragraph("Supermercado - Análisis Comercial", styles["Heading2"]))
    story.append(Spacer(1, 50))
    story.append(Paragraph("Autora: Fernanda Flores", styles["Normal"]))
    story.append(Paragraph("Instituto Beltrán - 2025", styles["Normal"]))
    story.append(PageBreak())
    
    # Resumen de tablas
    story.append(Paragraph("Tablas Cargadas", estilo_titulo))
    story.append(Spacer(1, 12))
    
    for nombre, df in dataframes.items():
        story.append(Paragraph(f"📊 {nombre}", estilo_subtitulo))
        story.append(Paragraph(f"   Columnas: {', '.join(df.columns)}", styles["Normal"]))
        story.append(Paragraph(f"   Registros: {len(df)}", styles["Normal"]))
        story.append(Spacer(1, 8))
    
    story.append(PageBreak())
    
    # Gráficos
    story.append(Paragraph("Análisis Gráfico", estilo_titulo))
    story.append(Spacer(1, 20))
    
    # Ordenar gráficos por nombre para mantener orden
    graficos_ordenados = sorted(graficos_combinados, key=lambda x: os.path.basename(x))
    
    descripciones = {
        "01_top10_productos_barras.png": "Top 10 Productos Más Vendidos - Gráfico de Barras",
        "02_top10_productos_torta.png": "Distribución de Top 10 Productos - Gráfico de Torta",
        "03_ventas_por_rubro_barras.png": "Ventas Totales por Rubro - Gráfico de Barras",
        "04_ventas_por_mes_lineal.png": "Ventas por Mes - Gráfico Lineal",
        "05_ticket_promedio_barras.png": "Ticket Promedio por Cliente - Gráfico de Barras",
        "06_facturas_mas_altas_barras.png": "Facturas Más Altas - Gráfico de Barras",
        "07_producto_mas_vendido_barras.png": "Producto Más Vendido en Cantidad - Gráfico de Barras",
        "08_ventas_por_cliente_lineal.png": "Ventas por Cliente - Gráfico Lineal",
        "09_ventas_por_cliente_cantidad_barras.png": "Cantidad de Ventas por Cliente - Gráfico de Barras"
    }
    
    for grafico in graficos_ordenados:
        nombre_archivo = os.path.basename(grafico)
        descripcion = descripciones.get(nombre_archivo, f"Gráfico: {nombre_archivo}")
        
        story.append(Paragraph(descripcion, estilo_subtitulo))
        story.append(Spacer(1, 10))
        
        try:
            img = Image(grafico, width=500, height=350)
            story.append(img)
            story.append(Spacer(1, 20))
        except Exception as e:
            story.append(Paragraph(f"Error al cargar imagen: {str(e)}", styles["Normal"]))
            story.append(Spacer(1, 20))
    
    doc.build(story)
    return pdf_path

# --- PROCESAMIENTO PRINCIPAL ---
print("=== INICIANDO PROCESAMIENTO ===")

# 1. Cargar todos los CSV con formato correcto
dataframes = cargar_tablas(archivos_csv)

# 2. Procesar CSV a Excel (individual y combinado)
print("\n=== CONVIRTIENDO CSV A EXCEL ===")
resumenes = []

try:
    with pd.ExcelWriter(salida_excel_combinado, engine="openpyxl") as writer_combinado:
        for nombre, df in dataframes.items():
            print(f" Procesando: {nombre}")
            
            # Guardar Excel individual
            ruta_excel_individual = os.path.join(carpeta_excels, f"{nombre}.xlsx")
            df.to_excel(ruta_excel_individual, index=False)
            print(f"   ✓ Excel individual: {ruta_excel_individual}")
            
            # Escribir en el Excel combinado
            # Limitar nombre de hoja a 31 caracteres (límite de Excel)
            nombre_hoja = nombre[:31]
            df.to_excel(writer_combinado, sheet_name=nombre_hoja, index=False)
            
            # Resumen estadístico
            resumen = df.describe(include="all").transpose()
            resumen["Archivo"] = nombre
            resumenes.append(resumen)

        # Guardar resumen general en el Excel combinado
        if resumenes:
            resumen_final = pd.concat(resumenes)
            resumen_final.to_excel(writer_combinado, sheet_name="Resumen_General")
            
    print(f" ✓ Excel combinado guardado: {salida_excel_combinado}")
    
except Exception as e:
    print(f"  Error guardando Excel combinado: {e}")

# 3. Generar gráficos combinados
print("\n=== GENERANDO GRÁFICOS COMBINADOS ===")
graficos_combinados = generar_graficos_combinados(dataframes, carpeta_graficos_combinados)

print(f"\n 📈 Se generaron {len(graficos_combinados)} gráficos combinados:")
for grafico in graficos_combinados:
    print(f"   ✓ {os.path.basename(grafico)}")

# 4. Crear informe PDF combinado
print("\n=== CREANDO INFORME COMBINADO ===")
if graficos_combinados:
    pdf_combinado = crear_informe_combinado(graficos_combinados, carpeta_informe_combinado, dataframes)
    print(f" ✓ Informe PDF generado: {pdf_combinado}")
else:
    print("  No se generaron gráficos para el informe")

print("\n" + "="*50)
print(" PROCESO FINALIZADO")
print("="*50)
print(f"  Carpeta con Excels individuales: {carpeta_excels}")
print(f"  Excel combinado: {salida_excel_combinado}")
print(f"  Gráficos combinados: {carpeta_graficos_combinados}")
print(f"  Informe PDF combinado: {carpeta_informe_combinado}")
print(f"  Total de gráficos generados: {len(graficos_combinados)}")

 Se encontraron 11 archivos CSV:
 - cliente.csv
 - cliente_mail.csv
 - cliente_telefono.csv
 - factura_detalle.csv
 - factura_enunciado.csv
 - localidad.csv
 - producto.csv
 - provincia.csv
 - rubros.csv
 - sucursal.csv
 - venta.csv
=== INICIANDO PROCESAMIENTO ===

 Cargando cliente...
   Columnas originales: ['id_cliente', 'nombre_cliente', 'direccion', 'telefono']
   Primeras filas:
   id_cliente nombre_cliente                    direccion    telefono
0           1     Juan Pérez    Av. Mitre 123, Avellaneda  1160001111
1           2    María López  Calle San Martín 456, Lanús  1160002222
2           3  Carlos García    Av. Belgrano 789, Quilmes  1160003333
   Tabla cliente procesada
   Columnas finales: ['id_cliente', 'nombre', 'apellido', 'direccion']
   Filas: 10

 Cargando cliente_mail...
   Columnas originales: ['id_email', 'id_cliente', 'email']
   Primeras filas:
   id_email  id_cliente                    email
0         1           1     juan_perez@gmail.com
1         2      

C:\Users\Fer\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\plotting\_matplotlib\core.py:1567: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax.set_xlim(left, right)


4. Generando ticket promedio...
5. Generando facturas más altas...
6. Generando producto más vendido...
7. Generando ventas por cliente...
8. Generando unión venta-cliente...

 📈 Se generaron 9 gráficos combinados:
   ✓ 01_top10_productos_barras.png
   ✓ 02_top10_productos_torta.png
   ✓ 03_ventas_por_rubro_barras.png
   ✓ 04_ventas_por_mes_lineal.png
   ✓ 05_ticket_promedio_barras.png
   ✓ 06_facturas_mas_altas_barras.png
   ✓ 07_producto_mas_vendido_barras.png
   ✓ 08_ventas_por_cliente_lineal.png
   ✓ 09_ventas_por_cliente_cantidad_barras.png

=== CREANDO INFORME COMBINADO ===
 ✓ Informe PDF generado: C:\Users\Fer\OneDrive\Escritorio\Modelado de Mineria de Datos\Proyecto_3\supermercado\Informe_Combinado\Informe_Combinado_Completo.pdf

🎉 PROCESO FINALIZADO
 📂 Carpeta con Excels individuales: C:\Users\Fer\OneDrive\Escritorio\Modelado de Mineria de Datos\Proyecto_3\supermercado\Excels_Individuales
 📊 Excel combinado: C:\Users\Fer\OneDrive\Escritorio\Modelado de Mineria de Datos\Proyect